In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

from langchain.callbacks import get_openai_callback

In [ ]:
set_llm_cache(InMemoryCache())

In [ ]:
from typing import Any, Dict, List


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples: List = examples

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        from random import choice
        return [choice(self.examples)]
    
    def add_example(self, example: Dict[str, str]) -> Any:
        return self.examples.append(example)

In [2]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(), 
    ]
)

In [ ]:
with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju?")
    print(usage)

In [3]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [5]:
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = ChatPromptTemplate.from_template(example_template)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. You give short answers"),
        example_prompt,
        ("human", "What do you know about {country}")
    ]
)

chain = final_prompt | chat 

chain.invoke(
    {
        "country": "Austrailia"
    }
)

I know this:
Capital: Canberra
Language: English
Famous Landmarks: Sydney Opera House, Great Barrier Reef
Currency: Australian Dollar

AIMessageChunk(content='I know this:\nCapital: Canberra\nLanguage: English\nFamous Landmarks: Sydney Opera House, Great Barrier Reef\nCurrency: Australian Dollar')